# Phase 3: Modeling & Inferencing

**Team**: The Outliers  
**Course**: Advanced Apex Project 1 - BITS Pilani Digital  
**Phase**: 3 (Model Construction & Evaluation)

---

## 🎯 What are we doing in Phase 3?

We are building **regression models** to predict house prices (`SalePrice`) using the features we engineered in Phase 2.

## 🤔 Why are we doing this?

- To **predict house prices accurately** for properties in Ames, Iowa
- To **understand which features** most influence property values
- To **compare different modeling approaches** (Simple vs Multiple Linear Regression)
- To **measure model performance** using statistical metrics

## 📊 Models to Build (as per Phase 3 requirements):

1. **Simple Linear Regression**: Uses ONE best feature to predict price
2. **Multiple Linear Regression**: Uses ALL features to predict price

## 📏 Evaluation Metrics:

- **R² (R-squared)**: How much variance in price our model explains (0 to 1, higher is better)
- **RMSE (Root Mean Squared Error)**: Average prediction error in dollars (lower is better)
- **MAE (Mean Absolute Error)**: Average absolute error in dollars (lower is better)

## 🔮 Expected Results:

From Phase 2, we know `Overall Qual` has 0.80 correlation with price:
- Simple LR should achieve R² ≈ 0.64
- Multiple LR should achieve R² > 0.70

---

## Step 1: Setup & Import Libraries

### 🎯 What are we doing?
Importing all necessary Python libraries for data manipulation, machine learning, and visualization.

### 🤔 Why these libraries?
- **pandas**: Load and work with datasets
- **numpy**: Mathematical operations
- **scikit-learn**: Build and evaluate machine learning models
- **matplotlib/seaborn**: Create visualizations

### 📊 Expected Result:
All libraries should import successfully with no errors.

---

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Settings
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Libraries imported successfully!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")

✅ Libraries imported successfully!
📦 Pandas version: 2.3.3
📦 NumPy version: 2.2.6


---

## Step 2: Load Engineered Dataset

### 🎯 What are we doing?
Loading the **engineered dataset** from Phase 2C (`AmesHousing_engineered.csv`).

### 🤔 Why this specific dataset?
This dataset contains:
- ✅ All missing values handled (imputed)
- ✅ All categorical features encoded (converted to numbers)
- ✅ New engineered features (Total_SF, Total_Bathrooms, etc.)
- ✅ Log-transformed skewed features for better modeling
- ✅ Multicollinearity reduced (highly correlated features removed)

### 📊 Expected Result:
- Dataset shape: **(2,930 rows × 71 columns)**
- Target variable: `SalePrice` (house prices in dollars)
- All features should be numeric (ready for machine learning)
- Minimal to no missing values

---

In [2]:
# Load the engineered dataset from Phase 2C
df = pd.read_csv("../data/AmesHousing_engineered.csv")

print("✅ Dataset loaded successfully!")
print(f"\n📊 Dataset Shape: {df.shape}")
print(f"   - Total Records (Houses): {df.shape[0]:,}")
print(f"   - Total Features (Columns): {df.shape[1]}")

# Check for missing values
missing_count = df.isnull().sum().sum()
print(f"\n❓ Missing Values: {missing_count}")
if missing_count > 0:
    print(f"   ⚠️ Warning: Found {missing_count} missing values - need to handle these")
else:
    print("   ✅ No missing values - dataset is complete!")

# Check data types
print(f"\n📋 Data Types:")
print(df.dtypes.value_counts())

# Display first few rows
print("\n📋 First 3 rows of the dataset:")
print(df.head(3))

✅ Dataset loaded successfully!

📊 Dataset Shape: (2930, 71)
   - Total Records (Houses): 2,930
   - Total Features (Columns): 71

❓ Missing Values: 3
   ⚠️ Warning: Found 3 missing values - need to handle these

📋 Data Types:
int64      58
float64    13
Name: count, dtype: int64

📋 First 3 rows of the dataset:
   Order        PID  MS SubClass  MS Zoning  Lot Frontage  Lot Area  \
0      1  526301100           20          5        141.00     31770   
1      2  526350040           20          4         80.00     11622   
2      3  526351010           20          5         81.00     14267   

   Lot Shape  Land Contour  Lot Config  Neighborhood  Condition 1  Bldg Type  \
0          0             3           0            15            2          0   
1          3             3           4            15            1          0   
2          0             3           0            15            2          0   

   House Style  Overall Qual  Overall Cond  Year Built  Year Remod/Add  \
0       

---

## Step 3: Investigate Missing Values

### 🎯 What are we doing?
Identifying exactly WHERE the 3 missing values are located (which columns and rows).

### 🤔 Why investigate first?
Before handling missing values, we need to understand:
- Which features contain missing data
- How many missing values per feature
- Whether these are critical features for modeling

### 📊 Expected Result:
- List of columns with missing values
- Count of missing values per column
- Decision on how to handle them (drop, impute, or investigate)

---

In [3]:
# Identify columns with missing values
missing_by_column = df.isnull().sum()
missing_features = missing_by_column[missing_by_column > 0]

print("🔍 Columns with Missing Values:")
print("="*50)
if len(missing_features) > 0:
    for col, count in missing_features.items():
        percentage = (count / len(df)) * 100
        print(f"   • {col}: {count} missing ({percentage:.2f}%)")
    
    print(f"\n📊 Total missing values: {missing_features.sum()}")
    
    # Show the rows with missing values
    print(f"\n📋 Rows containing missing values:")
    missing_rows = df[df.isnull().any(axis=1)]
    print(missing_rows[missing_features.index.tolist()].head(10))
else:
    print("   ✅ No missing values found!")

# Check data completeness
completeness = (1 - df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
print(f"\n✅ Dataset Completeness: {completeness:.4f}%")

🔍 Columns with Missing Values:
   • Lot Frontage: 3 missing (0.10%)

📊 Total missing values: 3

📋 Rows containing missing values:
      Lot Frontage
2256           NaN
2788           NaN
2892           NaN

✅ Dataset Completeness: 99.9986%


### 🔎 Analysis of Missing Values

**Finding**: All 3 missing values are in `Lot Frontage` column

**Context**:
- `Lot Frontage` = Linear feet of street connected to property
- Only 0.10% of data missing (3 out of 2,930 rows)
- Dataset is 99.9986% complete

**Next Step**: Investigate these 3 rows to understand their characteristics before deciding on handling strategy.

---

In [4]:
# Get the 3 rows with missing Lot Frontage
missing_indices = [2256, 2788, 2892]
missing_details = df.loc[missing_indices]

print("📋 Detailed view of rows with missing Lot Frontage:")
print("="*70)

# Show key identifying features
key_features = ['Order', 'PID', 'Neighborhood', 'Lot Area', 'Lot Frontage', 
                'Gr Liv Area', 'Overall Qual', 'SalePrice']

print(missing_details[key_features])

# Get statistics for Lot Frontage to understand typical values
print("\n\n📊 Lot Frontage Statistics (for context):")
print("="*70)
print(df['Lot Frontage'].describe())

# Check neighborhoods of missing rows
print("\n\n🏘️ Neighborhoods of rows with missing values:")
print("="*70)
for idx in missing_indices:
    neighborhood = df.loc[idx, 'Neighborhood']
    print(f"   Row {idx}: Neighborhood Code = {neighborhood}")

# Show Lot Frontage median by neighborhood for these specific neighborhoods
print("\n\n📐 Lot Frontage Median for these neighborhoods:")
print("="*70)
for idx in missing_indices:
    neighborhood = df.loc[idx, 'Neighborhood']
    neighborhood_median = df[df['Neighborhood'] == neighborhood]['Lot Frontage'].median()
    print(f"   Neighborhood {neighborhood}: Median = {neighborhood_median:.2f} feet")

📋 Detailed view of rows with missing Lot Frontage:
      Order        PID  Neighborhood  Lot Area  Lot Frontage  Gr Liv Area  \
2256   2257  916253320            10      9763           NaN         1502   
2788   2789  907230240            12      3612           NaN         1320   
2892   2893  916252170            10      8239           NaN         1295   

      Overall Qual  SalePrice  
2256             7     330000  
2788             6     137000  
2892             7     230000  


📊 Lot Frontage Statistics (for context):
count   2927.00
mean      69.46
std       21.73
min       21.00
25%       60.00
50%       70.00
75%       80.00
max      313.00
Name: Lot Frontage, dtype: float64


🏘️ Neighborhoods of rows with missing values:
   Row 2256: Neighborhood Code = 10
   Row 2788: Neighborhood Code = 12
   Row 2892: Neighborhood Code = 10


📐 Lot Frontage Median for these neighborhoods:
   Neighborhood 10: Median = nan feet
   Neighborhood 12: Median = nan feet
   Neighborhood 10: Media

### ⚠️ Issue Found: Neighborhood median calculation returned NaN

**Problem**: Need to properly calculate median by excluding NaN values

**Next**: Re-calculate neighborhood statistics correctly and determine best imputation strategy

---

In [5]:
# Properly investigate neighborhoods with missing values
print("🔍 Proper Neighborhood Analysis:")
print("="*70)

for idx in missing_indices:
    neighborhood = df.loc[idx, 'Neighborhood']
    lot_area = df.loc[idx, 'Lot Area']
    
    # Get all properties in this neighborhood (excluding current row)
    neighborhood_data = df[(df['Neighborhood'] == neighborhood) & (df.index != idx)]
    
    # Count properties in neighborhood
    total_in_neighborhood = len(neighborhood_data)
    
    # Get Lot Frontage values (excluding NaN)
    lot_frontage_values = neighborhood_data['Lot Frontage'].dropna()
    count_with_frontage = len(lot_frontage_values)
    
    # Calculate median (properly, excluding NaN)
    if count_with_frontage > 0:
        median_frontage = lot_frontage_values.median()
        mean_frontage = lot_frontage_values.mean()
        
        print(f"\n📍 Row {idx} (Neighborhood {neighborhood}):")
        print(f"   - Total properties in neighborhood: {total_in_neighborhood}")
        print(f"   - Properties with Lot Frontage data: {count_with_frontage}")
        print(f"   - Median Lot Frontage: {median_frontage:.2f} feet")
        print(f"   - Mean Lot Frontage: {mean_frontage:.2f} feet")
        print(f"   - This property's Lot Area: {lot_area:,.0f} sq ft")
    else:
        print(f"\n📍 Row {idx} (Neighborhood {neighborhood}):")
        print(f"   ⚠️ No other properties in this neighborhood have Lot Frontage data!")
        print(f"   Need to use overall median instead")

# Calculate overall median (excluding NaN) as fallback
overall_median = df['Lot Frontage'].median()
overall_mean = df['Lot Frontage'].mean()

print(f"\n\n📊 Overall Dataset Statistics (for fallback):")
print("="*70)
print(f"   Median Lot Frontage: {overall_median:.2f} feet")
print(f"   Mean Lot Frontage: {overall_mean:.2f} feet")

🔍 Proper Neighborhood Analysis:

📍 Row 2256 (Neighborhood 10):
   ⚠️ No other properties in this neighborhood have Lot Frontage data!
   Need to use overall median instead

📍 Row 2788 (Neighborhood 12):
   ⚠️ No other properties in this neighborhood have Lot Frontage data!
   Need to use overall median instead

📍 Row 2892 (Neighborhood 10):
   ⚠️ No other properties in this neighborhood have Lot Frontage data!
   Need to use overall median instead


📊 Overall Dataset Statistics (for fallback):
   Median Lot Frontage: 70.00 feet
   Mean Lot Frontage: 69.46 feet


---

## Step 4: Handle Missing Values

### 🎯 What are we doing?
Imputing the 3 missing `Lot Frontage` values using the **overall median** (70.00 feet).

### 🤔 Why overall median instead of neighborhood median?
- **Problem**: Neighborhoods 10 and 12 have NO other properties with Lot Frontage data
- **Solution**: Use the overall dataset median as the most reasonable estimate
- **Median vs Mean**: Using median (70.00) instead of mean (69.46) because median is more robust to outliers

### 📊 Expected Result:
- All 3 missing values filled with 70.00 feet
- Zero missing values in dataset
- Dataset ready for machine learning

---

In [6]:
# Handle missing values by imputing with overall median
print("🔧 Imputing Missing Lot Frontage Values:")
print("="*70)

# Store original count of missing values
original_missing = df['Lot Frontage'].isnull().sum()
print(f"Missing values BEFORE imputation: {original_missing}")

# Calculate overall median
overall_median = df['Lot Frontage'].median()
print(f"Imputation value (overall median): {overall_median:.2f} feet")

# Show which rows will be imputed
missing_indices = df[df['Lot Frontage'].isnull()].index.tolist()
print(f"\nRows to be imputed: {missing_indices}")

# Perform imputation
df['Lot Frontage'].fillna(overall_median, inplace=True)

# Verify imputation
remaining_missing = df['Lot Frontage'].isnull().sum()
print(f"\nMissing values AFTER imputation: {remaining_missing}")

# Verify total dataset completeness
total_missing = df.isnull().sum().sum()
print(f"\n✅ Total missing values in entire dataset: {total_missing}")

if total_missing == 0:
    print("   🎉 SUCCESS! Dataset is now 100% complete and ready for modeling!")
else:
    print(f"   ⚠️ Warning: {total_missing} missing values still remain")

# Show the imputed values
print(f"\n📋 Imputed values verification:")
print(df.loc[missing_indices, ['Order', 'Neighborhood', 'Lot Area', 'Lot Frontage', 'SalePrice']])

🔧 Imputing Missing Lot Frontage Values:
Missing values BEFORE imputation: 3
Imputation value (overall median): 70.00 feet

Rows to be imputed: [2256, 2788, 2892]

Missing values AFTER imputation: 0

✅ Total missing values in entire dataset: 0
   🎉 SUCCESS! Dataset is now 100% complete and ready for modeling!

📋 Imputed values verification:
      Order  Neighborhood  Lot Area  Lot Frontage  SalePrice
2256   2257            10      9763         70.00     330000
2788   2789            12      3612         70.00     137000
2892   2893            10      8239         70.00     230000


---

## Step 5: Prepare Features and Target Variable

### 🎯 What are we doing?
Separating the dataset into **features (X)** and **target variable (y)** for modeling.

### 🤔 Key Decisions:
- **Target Variable**: Use `SalePrice` (original price in dollars), not `SalePrice_Log`
- **Features**: All columns EXCEPT identifiers (`Order`, `PID`) and target variables
- **Columns to Exclude**: Order, PID, SalePrice, SalePrice_Log, Lot Area_Log (redundant with Lot Area)

### 📊 Expected Result:
- **X**: Feature matrix with ~66 features (all numeric)
- **y**: Target vector with 2,930 house prices
- Ready for train-test split

---

In [7]:
# Prepare features and target variable
print("🔧 Preparing Features and Target Variable:")
print("="*70)

# Define columns to exclude (identifiers and target variables)
columns_to_exclude = ['Order', 'PID', 'SalePrice', 'SalePrice_Log', 'Lot Area_Log']

print(f"Columns to exclude: {columns_to_exclude}")

# Create feature matrix X (all columns except excluded ones)
X = df.drop(columns=columns_to_exclude)

# Create target vector y (SalePrice in dollars)
y = df['SalePrice']

print(f"\n✅ Features (X):")
print(f"   - Shape: {X.shape}")
print(f"   - Total features: {X.shape[1]}")
print(f"   - Total records: {X.shape[0]:,}")

print(f"\n✅ Target (y):")
print(f"   - Shape: {y.shape}")
print(f"   - Total records: {y.shape[0]:,}")
print(f"   - Min price: ${y.min():,.0f}")
print(f"   - Max price: ${y.max():,.0f}")
print(f"   - Mean price: ${y.mean():,.0f}")
print(f"   - Median price: ${y.median():,.0f}")

print(f"\n📋 Feature names (first 10):")
for i, col in enumerate(X.columns[:10], 1):
    print(f"   {i}. {col}")

print(f"   ... and {X.shape[1] - 10} more features")

# Verify no missing values in X and y
print(f"\n🔍 Data Quality Check:")
print(f"   Missing values in X: {X.isnull().sum().sum()}")
print(f"   Missing values in y: {y.isnull().sum()}")
print(f"   ✅ Data is ready for modeling!")

🔧 Preparing Features and Target Variable:
Columns to exclude: ['Order', 'PID', 'SalePrice', 'SalePrice_Log', 'Lot Area_Log']

✅ Features (X):
   - Shape: (2930, 66)
   - Total features: 66
   - Total records: 2,930

✅ Target (y):
   - Shape: (2930,)
   - Total records: 2,930
   - Min price: $12,789
   - Max price: $755,000
   - Mean price: $180,796
   - Median price: $160,000

📋 Feature names (first 10):
   1. MS SubClass
   2. MS Zoning
   3. Lot Frontage
   4. Lot Area
   5. Lot Shape
   6. Land Contour
   7. Lot Config
   8. Neighborhood
   9. Condition 1
   10. Bldg Type
   ... and 56 more features

🔍 Data Quality Check:
   Missing values in X: 0
   Missing values in y: 0
   ✅ Data is ready for modeling!


---

## Step 6: Train-Test Split

### 🎯 What are we doing?
Splitting the dataset into **training set** (80%) and **testing set** (20%).

### 🤔 Why split the data?
- **Training set**: Used to build/train our regression models
- **Testing set**: Used to evaluate model performance on unseen data
- **80-20 split**: Standard practice that balances training data volume with testing reliability
- **random_state=42**: Ensures reproducibility (same split every time we run)

### 📊 Expected Result:
- X_train: 2,344 samples (80%)
- X_test: 586 samples (20%)
- y_train: 2,344 prices
- y_test: 586 prices

---

In [8]:
# Split data into training and testing sets
print("🔧 Splitting Data into Train and Test Sets:")
print("="*70)

# Perform train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.20,      # 20% for testing
    random_state=42      # For reproducibility
)

print(f"✅ Training Set:")
print(f"   - X_train shape: {X_train.shape}")
print(f"   - y_train shape: {y_train.shape}")
print(f"   - Number of samples: {X_train.shape[0]:,} ({(X_train.shape[0]/len(X))*100:.1f}%)")
print(f"   - Number of features: {X_train.shape[1]}")

print(f"\n✅ Testing Set:")
print(f"   - X_test shape: {X_test.shape}")
print(f"   - y_test shape: {y_test.shape}")
print(f"   - Number of samples: {X_test.shape[0]:,} ({(X_test.shape[0]/len(X))*100:.1f}%)")
print(f"   - Number of features: {X_test.shape[1]}")

print(f"\n📊 Target Variable Statistics:")
print(f"\n   Training Set (y_train):")
print(f"      - Mean price: ${y_train.mean():,.0f}")
print(f"      - Median price: ${y_train.median():,.0f}")
print(f"      - Min price: ${y_train.min():,.0f}")
print(f"      - Max price: ${y_train.max():,.0f}")

print(f"\n   Testing Set (y_test):")
print(f"      - Mean price: ${y_test.mean():,.0f}")
print(f"      - Median price: ${y_test.median():,.0f}")
print(f"      - Min price: ${y_test.min():,.0f}")
print(f"      - Max price: ${y_test.max():,.0f}")

print(f"\n✅ Data split complete! Ready to build models.")

🔧 Splitting Data into Train and Test Sets:
✅ Training Set:
   - X_train shape: (2344, 66)
   - y_train shape: (2344,)
   - Number of samples: 2,344 (80.0%)
   - Number of features: 66

✅ Testing Set:
   - X_test shape: (586, 66)
   - y_test shape: (586,)
   - Number of samples: 586 (20.0%)
   - Number of features: 66

📊 Target Variable Statistics:

   Training Set (y_train):
      - Mean price: $178,582
      - Median price: $160,000
      - Min price: $12,789
      - Max price: $755,000

   Testing Set (y_test):
      - Mean price: $189,651
      - Median price: $165,000
      - Min price: $44,000
      - Max price: $625,000

✅ Data split complete! Ready to build models.


---

## Step 7: Simple Linear Regression Model

### 🎯 What are we doing?
Building a **Simple Linear Regression** model using the **ONE best feature** to predict house prices.

### 🤔 Why Simple Linear Regression first?
- Establishes a **baseline** for model performance
- Shows how much predictive power a single feature has
- Easy to interpret and visualize
- From Phase 2, we know `Overall Qual` has the strongest correlation (0.80)

### 📊 Expected Result:
- Identify the best single feature (highest correlation with SalePrice)
- Build Simple LR model using only that feature
- Expected R² ≈ 0.64 (correlation² = 0.80² ≈ 0.64)

---

In [9]:
# Step 7a: Identify the best single feature
print("🔍 Identifying the Best Single Feature for Simple Linear Regression:")
print("="*70)

# Calculate correlations between all features and target variable
correlations = X_train.corrwith(y_train).abs().sort_values(ascending=False)

print(f"📊 Top 10 Features by Correlation with SalePrice:")
print("="*70)
for i, (feature, corr) in enumerate(correlations.head(10).items(), 1):
    print(f"   {i:2d}. {feature:25s} : {corr:.4f}")

# Select the best feature
best_feature = correlations.index[0]
best_correlation = correlations.iloc[0]

print(f"\n✅ Best Single Feature Selected:")
print(f"   Feature: {best_feature}")
print(f"   Correlation: {best_correlation:.4f}")
print(f"   Expected R²: {best_correlation**2:.4f} (correlation²)")

print(f"\n📋 This feature will be used for Simple Linear Regression")

🔍 Identifying the Best Single Feature for Simple Linear Regression:
📊 Top 10 Features by Correlation with SalePrice:
    1. Overall Qual              : 0.7953
    2. Gr Liv Area               : 0.6983
    3. Exter Qual                : 0.6853
    4. Kitchen Qual              : 0.6612
    5. Garage Cars               : 0.6439
    6. Total_Bathrooms           : 0.6391
    7. Total Bsmt SF             : 0.6126
    8. 1st Flr SF                : 0.6074
    9. Bsmt Qual                 : 0.6016
   10. Year Built                : 0.5454

✅ Best Single Feature Selected:
   Feature: Overall Qual
   Correlation: 0.7953
   Expected R²: 0.6325 (correlation²)

📋 This feature will be used for Simple Linear Regression


### 📈 Step 7b: Build and Train Simple Linear Regression

**What we're doing**: Training the model using only `Overall Qual` to predict `SalePrice`

**Model equation**: `SalePrice = β₀ + β₁ × Overall_Qual`

Where:
- β₀ = intercept (base price)
- β₁ = coefficient (price increase per quality point)

---

In [10]:
# Step 7b: Build and train Simple Linear Regression model
print("🏗️ Building Simple Linear Regression Model:")
print("="*70)

# Prepare data with only the best feature (Overall Qual)
X_train_simple = X_train[[best_feature]]  # Must be 2D array for sklearn
X_test_simple = X_test[[best_feature]]

print(f"Training data shape: {X_train_simple.shape}")
print(f"Testing data shape: {X_test_simple.shape}")

# Create and train the model
simple_lr = LinearRegression()
simple_lr.fit(X_train_simple, y_train)

print(f"\n✅ Model Trained Successfully!")
print(f"\n📊 Model Parameters:")
print(f"   Intercept (β₀): ${simple_lr.intercept_:,.2f}")
print(f"   Coefficient (β₁) for {best_feature}: ${simple_lr.coef_[0]:,.2f}")

print(f"\n💡 Interpretation:")
print(f"   Base Price: ${simple_lr.intercept_:,.0f}")
print(f"   For each 1-point increase in {best_feature}:")
print(f"      → Price increases by ${simple_lr.coef_[0]:,.0f}")

# Example prediction
example_quality = 7  # Average quality house
example_price = simple_lr.intercept_ + simple_lr.coef_[0] * example_quality
print(f"\n📋 Example: A house with {best_feature} = {example_quality}")
print(f"   Predicted Price: ${example_price:,.0f}")

🏗️ Building Simple Linear Regression Model:


Training data shape: (2344, 1)
Testing data shape: (586, 1)

✅ Model Trained Successfully!

📊 Model Parameters:
   Intercept (β₀): $-89,311.54
   Coefficient (β₁) for Overall Qual: $44,174.67

💡 Interpretation:
   Base Price: $-89,312
   For each 1-point increase in Overall Qual:
      → Price increases by $44,175

📋 Example: A house with Overall Qual = 7
   Predicted Price: $219,911


### 📊 Step 7c: Evaluate Simple Linear Regression Performance

**What we're evaluating**:
- **R² Score**: How much variance in price our model explains (0 to 1)
- **RMSE**: Average prediction error in dollars (lower is better)
- **MAE**: Average absolute error in dollars (lower is better)

**Evaluation on**: Both training set (to check fit) and testing set (to check generalization)

---

In [11]:
# Step 7c: Evaluate Simple Linear Regression model
print("📊 Evaluating Simple Linear Regression Model:")
print("="*70)

# Make predictions on training set
y_train_pred_simple = simple_lr.predict(X_train_simple)

# Make predictions on testing set
y_test_pred_simple = simple_lr.predict(X_test_simple)

# Calculate metrics for TRAINING set
train_r2_simple = r2_score(y_train, y_train_pred_simple)
train_rmse_simple = np.sqrt(mean_squared_error(y_train, y_train_pred_simple))
train_mae_simple = mean_absolute_error(y_train, y_train_pred_simple)

# Calculate metrics for TESTING set
test_r2_simple = r2_score(y_test, y_test_pred_simple)
test_rmse_simple = np.sqrt(mean_squared_error(y_test, y_test_pred_simple))
test_mae_simple = mean_absolute_error(y_test, y_test_pred_simple)

print(f"📈 TRAINING SET Performance:")
print(f"   R² Score    : {train_r2_simple:.4f} ({train_r2_simple*100:.2f}% variance explained)")
print(f"   RMSE        : ${train_rmse_simple:,.2f}")
print(f"   MAE         : ${train_mae_simple:,.2f}")

print(f"\n📉 TESTING SET Performance:")
print(f"   R² Score    : {test_r2_simple:.4f} ({test_r2_simple*100:.2f}% variance explained)")
print(f"   RMSE        : ${test_rmse_simple:,.2f}")
print(f"   MAE         : ${test_mae_simple:,.2f}")

# Check for overfitting/underfitting
r2_diff = abs(train_r2_simple - test_r2_simple)
print(f"\n🔍 Model Assessment:")
print(f"   R² difference (train - test): {r2_diff:.4f}")
if r2_diff < 0.05:
    print(f"   ✅ Good generalization! Model performs similarly on unseen data.")
elif r2_diff < 0.10:
    print(f"   ⚠️ Acceptable generalization with slight overfitting.")
else:
    print(f"   ⚠️ Significant overfitting detected!")

print(f"\n💡 Interpretation:")
print(f"   On average, predictions are off by ${test_mae_simple:,.0f}")
print(f"   The model explains {test_r2_simple*100:.1f}% of price variance using just ONE feature!")

📊 Evaluating Simple Linear Regression Model:
📈 TRAINING SET Performance:
   R² Score    : 0.6325 (63.25% variance explained)
   RMSE        : $46,744.71
   MAE         : $32,962.20

📉 TESTING SET Performance:
   R² Score    : 0.6512 (65.12% variance explained)
   RMSE        : $52,878.68
   MAE         : $36,141.27

🔍 Model Assessment:
   R² difference (train - test): 0.0187
   ✅ Good generalization! Model performs similarly on unseen data.

💡 Interpretation:
   On average, predictions are off by $36,141
   The model explains 65.1% of price variance using just ONE feature!


---

## Step 8: Multiple Linear Regression Model

### 🎯 What are we doing?
Building a **Multiple Linear Regression** model using **ALL 66 features** to predict house prices.

### 🤔 Why Multiple Linear Regression?
- Uses **ALL available information** (66 features vs just 1)
- Captures complex relationships between multiple factors
- Expected to perform better than Simple LR
- More realistic for real-world predictions

### 📊 Expected Result:
- Use all 66 features simultaneously
- Expected R² > 0.70 (better than Simple LR's 0.65)
- Lower prediction error (MAE)
- More accurate price predictions

---

In [12]:
# Step 8a: Build and train Multiple Linear Regression model
print("🏗️ Building Multiple Linear Regression Model:")
print("="*70)

print(f"Using ALL features:")
print(f"   Training data shape: {X_train.shape}")
print(f"   Testing data shape: {X_test.shape}")
print(f"   Total features: {X_train.shape[1]}")

# Create and train the model
multiple_lr = LinearRegression()
multiple_lr.fit(X_train, y_train)

print(f"\n✅ Model Trained Successfully!")
print(f"\n📊 Model Parameters:")
print(f"   Intercept (β₀): ${multiple_lr.intercept_:,.2f}")
print(f"   Number of coefficients: {len(multiple_lr.coef_)}")

# Show top 5 most influential features (by absolute coefficient value)
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': multiple_lr.coef_
})
feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values('Abs_Coefficient', ascending=False)

print(f"\n📊 Top 10 Most Influential Features:")
print("="*70)
for i, row in feature_importance.head(10).iterrows():
    print(f"   {row['Feature']:25s} : ${row['Coefficient']:>12,.2f}")

print(f"\n💡 These features have the largest impact on price predictions")

🏗️ Building Multiple Linear Regression Model:
Using ALL features:
   Training data shape: (2344, 66)
   Testing data shape: (586, 66)
   Total features: 66

✅ Model Trained Successfully!

📊 Model Parameters:
   Intercept (β₀): $2,623,545.75
   Number of coefficients: 66

📊 Top 10 Most Influential Features:
   Overall Qual              : $   12,305.28
   Garage Cars               : $   11,206.05
   Exter Qual                : $   10,175.53
   Kitchen Qual              : $    9,857.35
   Kitchen AbvGr             : $   -8,687.59
   Bsmt Cond                 : $   -7,943.73
   Bsmt Qual                 : $    7,647.23
   Overall Cond              : $    4,999.49
   Fireplaces                : $    4,957.09
   Bsmt Exposure             : $   -4,154.57

💡 These features have the largest impact on price predictions


### 📊 Step 8b: Evaluate Multiple Linear Regression Performance

**Evaluating**: Same metrics as Simple LR for direct comparison
- R² Score (variance explained)
- RMSE (root mean squared error)
- MAE (mean absolute error)

**Goal**: Multiple LR should outperform Simple LR by using all available features

---

In [13]:
# Step 8b: Evaluate Multiple Linear Regression model
print("📊 Evaluating Multiple Linear Regression Model:")
print("="*70)

# Make predictions on training set
y_train_pred_multiple = multiple_lr.predict(X_train)

# Make predictions on testing set
y_test_pred_multiple = multiple_lr.predict(X_test)

# Calculate metrics for TRAINING set
train_r2_multiple = r2_score(y_train, y_train_pred_multiple)
train_rmse_multiple = np.sqrt(mean_squared_error(y_train, y_train_pred_multiple))
train_mae_multiple = mean_absolute_error(y_train, y_train_pred_multiple)

# Calculate metrics for TESTING set
test_r2_multiple = r2_score(y_test, y_test_pred_multiple)
test_rmse_multiple = np.sqrt(mean_squared_error(y_test, y_test_pred_multiple))
test_mae_multiple = mean_absolute_error(y_test, y_test_pred_multiple)

print(f"📈 TRAINING SET Performance:")
print(f"   R² Score    : {train_r2_multiple:.4f} ({train_r2_multiple*100:.2f}% variance explained)")
print(f"   RMSE        : ${train_rmse_multiple:,.2f}")
print(f"   MAE         : ${train_mae_multiple:,.2f}")

print(f"\n📉 TESTING SET Performance:")
print(f"   R² Score    : {test_r2_multiple:.4f} ({test_r2_multiple*100:.2f}% variance explained)")
print(f"   RMSE        : ${test_rmse_multiple:,.2f}")
print(f"   MAE         : ${test_mae_multiple:,.2f}")

# Check for overfitting/underfitting
r2_diff_multiple = abs(train_r2_multiple - test_r2_multiple)
print(f"\n🔍 Model Assessment:")
print(f"   R² difference (train - test): {r2_diff_multiple:.4f}")
if r2_diff_multiple < 0.05:
    print(f"   ✅ Good generalization! Model performs similarly on unseen data.")
elif r2_diff_multiple < 0.10:
    print(f"   ⚠️ Acceptable generalization with slight overfitting.")
else:
    print(f"   ⚠️ Significant overfitting detected!")

print(f"\n💡 Interpretation:")
print(f"   On average, predictions are off by ${test_mae_multiple:,.0f}")
print(f"   The model explains {test_r2_multiple*100:.1f}% of price variance using ALL 66 features!")

📊 Evaluating Multiple Linear Regression Model:


📈 TRAINING SET Performance:
   R² Score    : 0.8607 (86.07% variance explained)
   RMSE        : $28,778.82
   MAE         : $18,341.38

📉 TESTING SET Performance:
   R² Score    : 0.8579 (85.79% variance explained)
   RMSE        : $33,755.05
   MAE         : $20,928.19

🔍 Model Assessment:
   R² difference (train - test): 0.0028
   ✅ Good generalization! Model performs similarly on unseen data.

💡 Interpretation:
   On average, predictions are off by $20,928
   The model explains 85.8% of price variance using ALL 66 features!


---

## Step 9: Model Comparison & Final Results

### 🎯 What are we comparing?
Side-by-side comparison of **Simple LR** vs **Multiple LR** performance

### 📊 Key Questions:
1. How much does using all features improve performance?
2. Which model generalizes better to unseen data?
3. What's the practical impact on prediction accuracy?

---

In [14]:
# Step 9: Model Comparison
print("📊 MODEL COMPARISON: Simple LR vs Multiple LR")
print("="*80)

# Create comparison dataframe
comparison = pd.DataFrame({
    'Metric': ['Features Used', 'R² (Train)', 'R² (Test)', 'RMSE (Test)', 'MAE (Test)', 'R² Difference'],
    'Simple LR': [
        f'1 ({best_feature})',
        f'{train_r2_simple:.4f}',
        f'{test_r2_simple:.4f}',
        f'${test_rmse_simple:,.0f}',
        f'${test_mae_simple:,.0f}',
        f'{abs(train_r2_simple - test_r2_simple):.4f}'
    ],
    'Multiple LR': [
        f'66 (All)',
        f'{train_r2_multiple:.4f}',
        f'{test_r2_multiple:.4f}',
        f'${test_rmse_multiple:,.0f}',
        f'${test_mae_multiple:,.0f}',
        f'{abs(train_r2_multiple - test_r2_multiple):.4f}'
    ]
})

print(comparison.to_string(index=False))

# Calculate improvements
r2_improvement = test_r2_multiple - test_r2_simple
mae_improvement = test_mae_simple - test_mae_multiple
mae_improvement_pct = (mae_improvement / test_mae_simple) * 100

print(f"\n\n🎯 KEY FINDINGS:")
print("="*80)
print(f"✅ R² Improvement: {r2_improvement:.4f} ({r2_improvement*100:.2f}% more variance explained)")
print(f"✅ MAE Improvement: ${mae_improvement:,.0f} ({mae_improvement_pct:.1f}% reduction in error)")
print(f"✅ Both models show EXCELLENT generalization (low train-test difference)")

print(f"\n\n💡 INTERPRETATION:")
print("="*80)
print(f"1. Simple LR (1 feature):")
print(f"   - Using ONLY 'Overall Qual', achieves R² = {test_r2_simple:.4f}")
print(f"   - Proves quality ratings are the strongest price predictor")
print(f"   - Average error: ${test_mae_simple:,.0f}")

print(f"\n2. Multiple LR (66 features):")
print(f"   - Using ALL available features, achieves R² = {test_r2_multiple:.4f}")
print(f"   - Explains {test_r2_multiple*100:.1f}% of price variance!")
print(f"   - Average error: ${test_mae_multiple:,.0f}")
print(f"   - Reduces prediction error by ${mae_improvement:,.0f} compared to Simple LR")

print(f"\n\n🏆 WINNER: Multiple Linear Regression")
print("="*80)
print(f"Multiple LR is the CLEAR WINNER:")
print(f"  • {r2_improvement*100:.1f}% better at explaining price variance")
print(f"  • ${mae_improvement:,.0f} lower average prediction error")
print(f"  • Still maintains excellent generalization to new data")
print(f"  • Ready for production use in price prediction!")

print(f"\n✅ Phase 3 Complete! Both models successfully built and evaluated.")

📊 MODEL COMPARISON: Simple LR vs Multiple LR
       Metric        Simple LR Multiple LR
Features Used 1 (Overall Qual)    66 (All)
   R² (Train)           0.6325      0.8607
    R² (Test)           0.6512      0.8579
  RMSE (Test)          $52,879     $33,755
   MAE (Test)          $36,141     $20,928
R² Difference           0.0187      0.0028


🎯 KEY FINDINGS:
✅ R² Improvement: 0.2066 (20.66% more variance explained)
✅ MAE Improvement: $15,213 (42.1% reduction in error)
✅ Both models show EXCELLENT generalization (low train-test difference)


💡 INTERPRETATION:
1. Simple LR (1 feature):
   - Using ONLY 'Overall Qual', achieves R² = 0.6512
   - Proves quality ratings are the strongest price predictor
   - Average error: $36,141

2. Multiple LR (66 features):
   - Using ALL available features, achieves R² = 0.8579
   - Explains 85.8% of price variance!
   - Average error: $20,928
   - Reduces prediction error by $15,213 compared to Simple LR


🏆 WINNER: Multiple Linear Regression
Multipl